In [1]:
!pip3 install hdbscan 
!pip3 install bertopic

In [2]:
import spacy
import pandas as pd

import json
import spacy
import requests
import random
import time

import spacy
from tqdm import tqdm 
from collections import Counter

import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

import pyLDAvis
import pyLDAvis.gensim

import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/express/imshow_utils.py:24: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  np.bool8: (False, True),
/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/graph_objs/__init__.py:288: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):
/Users/lydiafield/anaconda3/

In [46]:
blue_states = pd.read_csv('/Users/lydiafield/Desktop/Semestre 1/Digital Methods/blue_states.csv')

In [54]:
docs=list(blue_states['Hit Sentence'])
docs=[str(i) for i in docs]
model = SentenceTransformer("all-MiniLM-L6-v2")
document_vectors = model.encode(docs,show_progress_bar=True)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.01, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=10, min_samples=5, metric='euclidean', cluster_selection_method='eom')
vectorizer = CountVectorizer(stop_words='english')
topic_model = BERTopic(
    embedding_model=model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer,
    verbose=True
).fit(docs, document_vectors)
bar_topic = topic_model.visualize_barchart(top_n_topics=20,n_words=10, title='Blue States Topics')

Batches:   0%|          | 0/155 [00:00<?, ?it/s]

2023-12-12 01:52:56,162 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2023-12-12 01:53:04,283 - BERTopic - Dimensionality - Completed ✓
2023-12-12 01:53:04,286 - BERTopic - Cluster - Start clustering the reduced embeddings
2023-12-12 01:53:04,360 - BERTopic - Cluster - Completed ✓
2023-12-12 01:53:04,362 - BERTopic - Representation - Extracting topics from clusters using representation models.
2023-12-12 01:53:04,527 - BERTopic - Representation - Completed ✓


In [55]:
bar_topic

/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

/Users/lydiafield/anaconda3/lib/python3.11/site-packages/plotly/io/_renderers.py:395: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [50]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1455,-1_books_book_banning_banned,"[books, book, banning, banned, people, school,...","[""I read banned books"" No you don't, @timesof..."
1,0,316,0_illinois_pritzker_state_gov,"[illinois, pritzker, state, gov, nation, libra...","[ @CalltoActivism: THANK YOU , Governor Pritzk..."
2,1,127,1_banned_book_read_banning,"[banned, book, read, banning, stories, books, ...","[ @MoveOn: “Kamau, would you like to wear a t-..."
3,2,110,2_california_newsom_gavin_ca,"[california, newsom, gavin, ca, tonythurmond, ...",[ @MikeSington: Gavin Newsom is anything but a...
4,3,103,3_moms_liberty_hitler_group,"[moms, liberty, hitler, group, newsletter, quo...",[ @NoLieWithBTC: This group is very bad news. ...
...,...,...,...,...,...
133,132,10,132_red_states_education_teachers,"[red, states, education, teachers, wux1nshxpp,...",[ @TheHeartlandPOD: If you have any doubt abou...
134,133,10,133_judy_blume_adult_fiction,"[judy, blume, adult, fiction, unless, qu7exxuh...",[ @mechanicalkurt: judy blume is the 🐐 ; Start...
135,134,10,134_buddy_haul_nearest_asap,"[buddy, haul, nearest, asap, stephenking, king...",[ @StephenKing: Do it! You’ll learn powerful a...
136,135,10,135_hoax_insurance_florida_basically,"[hoax, insurance, florida, basically, flood, g...",[ @atrupar: At this point he should just chang...


In [51]:
import pandas as pd
import tqdm

In [57]:
corpus_text = blue_states['Hit Sentence'].values
corpus_text = [str(e) for e in corpus_text]

In [58]:
nlp = spacy.load("en_core_web_sm")
documents_nlp = []
for d in tqdm.tqdm(nlp.pipe(corpus_text,n_process=5),total=len(corpus_text)):
    documents_nlp.append(d)

  0%|                                                  | 0/4942 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

100%|██████████████████████████████████████| 4942/4942 [00:18<00:00, 269.23it/s]


In [59]:
docs2 = [[token.lemma_.lower() for token in doc if  token.is_alpha] for doc in documents_nlp]
docs2 = [[token for token in doc if len(token) > 3] for doc in docs2]

In [60]:
id2word = Dictionary(docs2)
id2word.filter_extremes(no_below=20, no_above=0.1)

In [61]:
corpus = [id2word.doc2bow(text) for text in docs2]

In [62]:
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=10,
                   passes=20)

In [63]:
label_dict = {i: ', '.join([token for token, score in lda_model.show_topic(i, topn=10)]) for i in range(0, lda_model.num_topics)}
for topic in label_dict:
    print (topic,':\t', label_dict[topic])

0 :	 author, week, against, books, censorship, board, read, fight, banned, county
1 :	 want, history, more, back, right, desantis, will, their, like, think
2 :	 year, adult, very, just, liberty, like, post, nationwide, name, these
3 :	 right, their, republicans, speech, free, your, take, abortion, vote, party
4 :	 illinois, first, sign, public, pritzker, nation, bill, become, country, governor
5 :	 drag, black, history, still, child, protect, show, your, want, kill
6 :	 their, student, parent, public, include, child, america, right, make, education
7 :	 anti, bill, pass, house, public, city, look, republicans, matter, over
8 :	 mass, assault, shooting, will, republicans, more, weapon, should, child, keep
9 :	 want, read, california, there, will, child, should, into, parent, your


In [64]:
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=lda_model, corpus=corpus, coherence='u_mass')
cm.get_coherence()

-3.323616929069228

In [65]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, id2word, sort_topics=False)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.027399 -0.135868       1        1  11.753153
1     -0.053871  0.015009       2        1  12.460102
2     -0.157233 -0.089344       3        1   8.069397
3     -0.008875  0.081656       4        1  13.915948
4      0.308014 -0.202148       5        1   7.408401
5     -0.160215 -0.079368       6        1   8.278393
6     -0.031747  0.039179       7        1  10.691490
7      0.176641  0.213675       8        1   8.134239
8     -0.020009  0.149024       9        1   9.466301
9     -0.025306  0.008183      10        1   9.822575, topic_info=         Term        Freq       Total Category  logprob  loglift
58   illinois  379.000000  379.000000  Default  30.0000  30.0000
157     first  282.000000  282.000000  Default  29.0000  29.0000
128      want  628.000000  628.000000  Default  28.0000  28.0000
286   history  416.000000  416.000000  Default  27.0000  27.0000
464     black  220.000000  220.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
75      their   69.886056  552.505585  Topic10  -4.3191   0.2529
82    because   55.593816  267.379820  Topic10  -4.5479   0.7499
42       just   62.245376  423.768590  Topic10  -4.4349   0.4024
18      would   48.092480  218.684344  Topic10  -4.6929   0.8060
119      stop   44.520947  214.640639  Topic10  -4.7700   0.7475

[559 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
347      10  0.989897      able
397       4  0.668355  abortion
397       9  0.328873  abortion
329       1  0.156412    access
329       4  0.270713    access
...     ...       ...       ...
122       4  0.343629      your
122       6  0.209165      your
122      10  0.201695      your
54        1  0.206468      zero
54        9  0.784577      zero

[1282 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [66]:
docs_2_topics = lda_model[corpus]
topic_2_docs={}
for i, doc_2_topics in enumerate(docs_2_topics):
    for (topic,strength) in doc_2_topics:
        topic_2_docs.setdefault(topic,[]).append((i,strength))

In [67]:
topic_number=9
best_doc = sorted(topic_2_docs[topic_number], key=lambda item: item[1], reverse=True)
print(label_dict[topic_number])
for doc in best_doc[:10]:
    print (corpus_text[doc[0]],doc[1])

want, read, california, there, will, child, should, into, parent, your
 @MikeSington: Gavin Newsom is anything but a promoter of freedom. ; California passes “anti-book ban” legislation. It would fine school districts for rejecting textbooks or library books for discriminatory reasons. Governor Gavin Newsom: “California is the true freedom state: a place where families — not political fanatics — have the freedom to…   /8YkuCnGZKg 0.96537805
 @againstgrmrs: Reminder: These are the books that California's Governor want to fine schools for banning...   /Q5Uy8Mo9XY ;   /vNyaTopgEp BREAKING: California will now fine schools that ban pornographic books from their libraries and classrooms. Gavin Newsom said that people who think kids shouldn't have access to them are "fringe ideologues." We say anyone who does needs to have their… 0.95713633
 @CAgovernor: Translation: California will put porn in the hands of your child. ; California is banning book bans and textbook censorship in our schools.